# Time Evolution 

This jupyter notebook is for the time evolution of a neels state using heisenberg hamiltonian.

Heisenberg Hamiltonian is given by: 

$\hat{H} = \sum \limits_{i=1}^{N-1} S_{j}^zS_{j+1}^z + \frac{1}{2} S^{+}_{j}S^{-}_{j+1} + \frac{1}{2} S^{-}_{j}S^{+}_{j+1}$


## Initial state:

$| \psi_0 \rangle = | 0 \rangle |1 \rangle \ldots | 0 \rangle |1 \rangle $

## TEBD (time-evolution block-decimation) Method : 

The time evolved quantum state, $|\psi(t) \rangle = U |\psi_0 \rangle = e^{iHt} | \psi_0 \rangle$. 

If H is given by $ H = A +B$ then, 

\begin{equation}\label{exp_A_B}
    e^{i(A+B)t} = e^{iAt} e^{iBt} e^{i([A,B])t}
\end{equation}

we can approximate this by product of evolution by each term in the hamiltonian.
\begin{equation}
     e^{i(A+B)t} \sim e^{iAt} e^{iBt} 
\end{equation}

The error in this can be approximated by:

\begin{equation}
    \begin{split}
         e^{i(A+B)t}  &= \mathbb{I} + i(A+B)t - \frac{(A+B)^2t^2}{2!} 
 - \frac{i(A+B)^3t^3}{3!} + \ldots\\
 &=\mathbb{I} + i(A+B)t - \frac{(A^2 + B^2 + AB + BA) t^2}{2!} 
 - \frac{i(A+B)^3t^3}{3!} + \ldots\\
 e^{iAt} e^{iBt} & = \left[ \mathbb{I} + iAt - \frac{A^2 t^2}{2!} -i \frac{A^3 t^3}{3!} + \ldots \right] \left[ \mathbb{I} + iBt - \frac{B^2 t^2}{2!} -i \frac{B^3 t^3}{3!} + \ldots \right]\\
 & = \mathbb{I} + i(A+B)t -\frac{(A^2 + B^2 + 2AB)t^2}{2!} + \mathcal{O}(t^3)\\
  e^{i(A+B)t} -  e^{iAt} e^{iBt} &= \frac{\left[ A,B\right]t^2}{2!} + \mathcal{O}(t^3)
    \end{split}
\end{equation}

So, the error depends on the commutator $\left[ A, B \right]$ and also on the $t^2 $. so we can reduce the error we approximate the evolved state after very small time $\epsilon$. We can break the time evolution in n steps such that $t = n\epsilon$

So, \begin{equation}
\begin{split}
     e^{i(A+B)t} &= e^{i(A+B)\epsilon}e^{i(A+B)\epsilon}e^{i(A+B)\epsilon} \ldots\\
     e^{i(A+B)t} &= \left[ e^{i(A+B)\epsilon} \right]^n\\
      e^{i(A+B)t} & = \left[ e^{iAt/n} e^{iBt/n} \right]^n
\end{split}
\end{equation}
Here, n is the number of steps. 

There are two main ways of decomposing into two site unitary operator :



$\begin{aligned}[t]
        U = \mathrm{e}^{-i\frac{\delta t}{2}H_{1,2}} \mathrm{e}^{-i\frac{\delta t}{2}H_{2,3}}...\mathrm{e}^{-i\frac{\delta t}{2}H_{N-1,N}}\mathrm{e}^{-i\frac{\delta t}{2}H_{N-1,N}}\mathrm{e}^{-i\frac{\delta t}{2}H_{N-2,N-1}}....\mathrm{e}^{-i\frac{\delta t}{2}H_{1,2}} + O(\delta t)^{3}\\
    \end{aligned}$
  
  
$H_{1,2} , H_{2,3} \ldots $ are the trotter gates acting on sites (1,2) and (2,3) and so on.. and then apply it in reverse. this full process will be equivalent to time-evolution by one time step.



In [1]:
using ITensors
using DelimitedFiles

In [2]:
# I: Initialise state:

# Firsly we define the number of particles
L = 10

# define the site indices for spin-1/2 particles: 
s = siteinds("S=1/2",L)

# Initial MPS state can be created as :
states = [isodd(n) ? "Up" : "Dn" for n in 1:L,conserve_qns=true]
ψ0 = (MPS(s,states));  # t=0 state
normalize!(ψ0);

partition = floor(Int,L/2);

@show ψ0

ψ0 = MPS
[1] ((dim=2|id=216|"S=1/2,Site,n=1"), (dim=1|id=432|"Link,l=1"))
[2] ((dim=1|id=432|"Link,l=1"), (dim=2|id=195|"S=1/2,Site,n=2"), (dim=1|id=125|"Link,l=2"))
[3] ((dim=1|id=125|"Link,l=2"), (dim=2|id=23|"S=1/2,Site,n=3"), (dim=1|id=179|"Link,l=3"))
[4] ((dim=1|id=179|"Link,l=3"), (dim=2|id=552|"S=1/2,Site,n=4"), (dim=1|id=789|"Link,l=4"))
[5] ((dim=1|id=789|"Link,l=4"), (dim=2|id=340|"S=1/2,Site,n=5"), (dim=1|id=622|"Link,l=5"))
[6] ((dim=1|id=622|"Link,l=5"), (dim=2|id=238|"S=1/2,Site,n=6"), (dim=1|id=132|"Link,l=6"))
[7] ((dim=1|id=132|"Link,l=6"), (dim=2|id=948|"S=1/2,Site,n=7"), (dim=1|id=712|"Link,l=7"))
[8] ((dim=1|id=712|"Link,l=7"), (dim=2|id=674|"S=1/2,Site,n=8"), (dim=1|id=797|"Link,l=8"))
[9] ((dim=1|id=797|"Link,l=8"), (dim=2|id=491|"S=1/2,Site,n=9"), (dim=1|id=64|"Link,l=9"))
[10] ((dim=1|id=64|"Link,l=9"), (dim=2|id=136|"S=1/2,Site,n=10"))



MPS
[1] ((dim=2|id=216|"S=1/2,Site,n=1"), (dim=1|id=432|"Link,l=1"))
[2] ((dim=1|id=432|"Link,l=1"), (dim=2|id=195|"S=1/2,Site,n=2"), (dim=1|id=125|"Link,l=2"))
[3] ((dim=1|id=125|"Link,l=2"), (dim=2|id=23|"S=1/2,Site,n=3"), (dim=1|id=179|"Link,l=3"))
[4] ((dim=1|id=179|"Link,l=3"), (dim=2|id=552|"S=1/2,Site,n=4"), (dim=1|id=789|"Link,l=4"))
[5] ((dim=1|id=789|"Link,l=4"), (dim=2|id=340|"S=1/2,Site,n=5"), (dim=1|id=622|"Link,l=5"))
[6] ((dim=1|id=622|"Link,l=5"), (dim=2|id=238|"S=1/2,Site,n=6"), (dim=1|id=132|"Link,l=6"))
[7] ((dim=1|id=132|"Link,l=6"), (dim=2|id=948|"S=1/2,Site,n=7"), (dim=1|id=712|"Link,l=7"))
[8] ((dim=1|id=712|"Link,l=7"), (dim=2|id=674|"S=1/2,Site,n=8"), (dim=1|id=797|"Link,l=8"))
[9] ((dim=1|id=797|"Link,l=8"), (dim=2|id=491|"S=1/2,Site,n=9"), (dim=1|id=64|"Link,l=9"))
[10] ((dim=1|id=64|"Link,l=9"), (dim=2|id=136|"S=1/2,Site,n=10"))


In [3]:
# II: The Time evolution operator :

dt =0.01
τ = -1im*dt # -it

# Now that we have out initial state, we need to create the time evolution operator due to Hamiltonian H
#Time evolved state is found by using TEBD technique, where we apply the time evolution U_{eff} 
#to only 2 site at a time and repeat this for all site pairs
# Here, we create a function where we define firstly the Operator name and then the site type where we have to be 
#spin 1/2 following that we define the two site indices. and finally exponentiate it.


function ITensors.op(::OpName"expH",::SiteType"S=1/2",s1::Index, s2::Index)
    h =  op("Sz", s1) * op("Sz", s2) + 0.5*op("S+",s1)*op("S-",s2) + 0.5*op("S-",s1)*op("S+",s2)
  return exp((τ/2)*h)
end
 

In [4]:
# We can make an array of strings and convert it into an array of all the needed 2 site trotter gates

os =[]  # create an empty array to store strings of operators and the corresponding sites

for i=1:(L-1)
    q = ("expH",i,i+1) #call the function which is exp{-ih_{ij}t}
    push!(os,q)        #add the tuple to the empty array
end

trotter_gate = ops(s,os)    #convert os containing strings of operator into trotter gates i.e order-4 tenso
                                #reapply the gates but in reverse order from right to left in chain
append!(trotter_gate,reverse(trotter_gate))


18-element Vector{ITensor}:
 ITensor ord=4
Dim 1: (dim=2|id=216|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=195|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=216|"S=1/2,Site,n=1")'
Dim 4: (dim=2|id=195|"S=1/2,Site,n=2")'
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2×2
[:, :, 1, 1] =
 0.9999992187501016 - 0.001249999674479192im  0.0 + 0.0im
                0.0 + 0.0im                   0.0 + 0.0im

[:, :, 2, 1] =
                0.0 + 0.0im                   …  3.1249959309915118e-6 - 0.002499995442711436im
 0.9999960937541708 + 0.001249995768232244im                       0.0 + 0.0im

[:, :, 1, 2] =
                   0.0 + 0.0im                    …  0.9999960937541706 + 0.0012499957682322437im
 3.1249959309915114e-6 - 0.0024999954427114357im                    0.0 + 0.0im

[:, :, 2, 2] =
 0.0 + 0.0im                 0.0 + 0.0im
 0.0 + 0.0im  0.9999992187501016 - 0.001249999674479192im
 ITensor ord=4
Dim 1: (dim=2|id=195|"S=1/2,Site,n=2")
Dim 2: (dim=2|id=23|"S=1/2,Site,n=3")
Dim 3: (dim=2|id=19

In [5]:
#parameters:

time_steps =10
io = open("Itensor_entropy_L_"*string(L)*".txt","w")
psi = copy(ψ0)

for t=1:time_steps

    # Evolution by Hamiltonian :
    psi = apply(trotter_gate,psi;cutoff=1E-8) #simple time evolution e^{-iHt}
    normalize!(psi)

    # Entanglement Entropy:

    # First orthogonalise the MPS to the site L/2, we want to do the SVD of this matrix , 
    # so we will make the physical index at site L/2 (index j) and the the left bond (index k) as the rows of the matrix 
    # and then do the SVD to get the singular matrix

    orthogonalize!(psi,partition)
    j = siteind(psi,partition)
    k = linkind(psi,partition-1)

    #To find the SVD

    U,S,V = svd(psi[partition],(j,k),cutoff=1E-8,maxdim=1024)

    Von_Entropy = 0
    # We convert the S matrix to array:
    S_matrix = Array(S,inds(S)) 

    Von_Entropy = 0

    for n=1:size(S_matrix,1)

        p = S[n,n]^2
        round(p;digits=6)
        Von_Entropy -= p * log(p)
    end

    writedlm(io,Von_Entropy)
    println(Von_Entropy)
    flush(io)

end


0.0002899067254205215
0.0010209062405303418
0.002114275106556353
0.003527869807726758
0.005232063435789942
0.007203830956231167
0.009424246735278871
0.011877180210136284
0.014548527954543495
0.01742572318351082
